# 2.3.2 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought
* Best result: A2

In [13]:
from utils.parser import Parser
from utils.openai import OpenAI

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()

In [14]:
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-01-TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print(len(pdf_prompts))

10


In [15]:
# A.1) Ein Thread – Analyse und Extraktion in einem Schritt
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 
msg = ava.request([
    *pdf_prompts,
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
        # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
    }
])

print(msg)

### Art der baulichen Nutzung:
1.1 Art der baulichen Nutzung (§§ 1 - 15 BauNVO)
1.1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
1.1.1.1 Zulässig sind / zulässig ist:
1.1.1.1.1 Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
1.1.1.1.2 Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
1.1.1.2 Nicht zulässig sind / nicht zulässig ist:
1.1.1.2.1 Nutzungen gemäß § 8 (3) Nr.3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
1.1.1.2.2 Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) BauNVO i.V